In [0]:
from google.colab import drive
drive.mount('/content/grdive')

Drive already mounted at /content/grdive; to attempt to forcibly remount, call drive.mount("/content/grdive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import math 
from collections import deque
from numpy import array
import random
import datetime
import calendar
import time

In [0]:
df =pd.read_excel('/content/grdive/My Drive/CFD Machine Learning/CFD.xlsx')

In [0]:
df.head()

,Pressure,Density,Temperature,ccx,ccy,ccz,U1,U2,U3
0,99999.401,1.186296,293.0,-15.66665,-15.66665,0.333333,1.000053,0.000269,0.000074
1,99999.401,1.186296,293.0,-15.00000,-15.66665,0.333333,1.000582,0.000954,0.000260
2,99999.400,1.186296,293.0,-14.33335,-15.66665,0.333333,1.001449,0.001724,0.000427
3,99999.399,1.186296,293.0,-13.66665,-15.66665,0.333333,1.002579,0.002579,0.000593
4,99999.397,1.186296,293.0,-13.00000,-15.66665,0.333333,1.003905,0.003499,0.000760


In [0]:
df=df.iloc[:,3:9]

In [0]:
df.head()

,ccx,ccy,ccz,U1,U2,U3
0,-15.66665,-15.66665,0.333333,1.000053,0.000269,0.000074
1,-15.00000,-15.66665,0.333333,1.000582,0.000954,0.000260
2,-14.33335,-15.66665,0.333333,1.001449,0.001724,0.000427
3,-13.66665,-15.66665,0.333333,1.002579,0.002579,0.000593
4,-13.00000,-15.66665,0.333333,1.003905,0.003499,0.000760


In [0]:
print("X min "+str(min(df.iloc[:,0])))
print("X max "+str(max(df.iloc[:,0])))
print("Y min "+str(min(df.iloc[:,1])))
print("Y max "+str(max(df.iloc[:,1])))
print("Z min "+str(min(df.iloc[:,2])))
print("Z max "+str(max(df.iloc[:,2])))

X min -15.66665
X max 47.66665
Y min -15.666651
Y max 15.666651
Z min 0.1652128
Z max 39.66665


In [0]:
df['distance']=np.sqrt((df['ccx']**2)+(df['ccy']**2)+(df['ccz']**2))

In [0]:
df.head()

,ccx,ccy,ccz,U1,U2,U3,distance
0,-15.66665,-15.66665,0.333333,1.000053,0.000269,0.000074,22.158496
1,-15.00000,-15.66665,0.333333,1.000582,0.000954,0.000260,21.692281
2,-14.33335,-15.66665,0.333333,1.001449,0.001724,0.000427,21.236760
3,-13.66665,-15.66665,0.333333,1.002579,0.002579,0.000593,20.792603
4,-13.00000,-15.66665,0.333333,1.003905,0.003499,0.000760,20.360625


In [0]:
df=df.sort_values('distance')

In [0]:
df.head()

,ccx,ccy,ccz,U1,U2,U3,distance
276101,-3.549367,0.873805,0.165214,0.157078,0.312564,0.086993,3.659076
276075,-3.549369,-0.873805,0.165215,0.186573,0.425258,0.081138,3.659077
276008,-0.873900,-3.549809,0.165215,1.250538,0.322960,0.072068,3.659527
276166,-0.873901,3.549815,0.165213,1.157566,0.301015,0.068508,3.659534
2717,0.873998,3.550097,0.165215,0.683874,0.066385,0.019641,3.659830


In [0]:
temp=df.iloc[:,[0,1,2,6,3,4,5]].values

In [0]:
temp.shape

(346167, 7)

In [0]:
sequential_data=[]
nearby_velocities =deque(maxlen=501)
temp_data=deque(maxlen=501)

In [0]:
#Creates the complete sequential data to be split into train and test sequences
for i in range(0,len(temp)-1):
  nearby_velocities.append([n for n in temp[i][0:4]])
  temp_data.append([n for n in temp[i][4:7]])
  if len(nearby_velocities) ==501:
      sequential_data.append([np.array(nearby_velocities),np.array(temp_data[250])])
  

In [0]:
sequential_data=sequential_data[0:55000]

In [0]:
sequential_data[10]

[array([[-1.8385877 , -3.1714118 ,  0.16549186,  3.66955653],
        [-1.8385861 ,  3.1714135 ,  0.16549133,  3.66955717],
        [ 1.8389244 , -3.1719485 ,  0.16549151,  3.67018905],
        ...,
        [ 0.16395774,  4.0793797 ,  0.50014753,  4.11319443],
        [ 0.16395787, -4.0793821 ,  0.5001475 ,  4.11319681],
        [ 4.0801651 , -0.16395922,  0.50015113,  4.11397387]]),
 array([0.65150939, 0.65397628, 0.044262  ])]

In [0]:
random.shuffle(sequential_data)

In [0]:
sequential_data[10]

[array([[-4.166587  ,  1.1666675 ,  8.8324791 ,  9.835357  ],
        [ 1.1666688 , -4.1665985 ,  8.8324775 ,  9.83536059],
        [-1.166669  ,  4.166599  ,  8.8324787 ,  9.8353619 ],
        ...,
        [ 1.166685  , -0.50001145,  9.8312707 ,  9.91287289],
        [ 1.166685  ,  0.50001158,  9.8312708 ,  9.91287299],
        [ 9.4983692 ,  1.832955  ,  2.1666575 ,  9.9132813 ]]),
 array([0.84183206, 0.31399956, 0.03522027])]

In [0]:
print(len(sequential_data))

55000


In [0]:
X_temp=array([n[0] for n in sequential_data[0:55000]])
Y_temp=array([n[1] for n in sequential_data[0:55000]])

In [0]:
print(X_temp.shape)
print(Y_temp.shape)

(55000, 501, 4)
(55000, 3)


In [0]:
print(X_temp.shape)
print(Y_temp.shape)

(55000, 501, 4)
(55000, 3)


In [0]:
#GET SPECIFIC INDEX
train_index=[]
validate_index=[]
for i in range(1, 55001):
  if(i%11)==0:
    validate_index.append(i)
  else:
    train_index.append(i) 

In [0]:
print(len(train_index))
print(len(validate_index))

50000
5000


In [0]:
print(min(train_index))
print(min(validate_index))
print(max(train_index))
print(max(validate_index))

1
11
54999
55000


In [0]:
X_train = np.copy(X_temp[train_index, ])
Y_train=np.copy(Y_temp[train_index, ])

In [0]:
X_validate = np.copy(X_temp[validate_index[:-1], ])
Y_validate = np.copy(Y_temp[validate_index[:-1], ])

In [0]:
#DONE TILL ABOVE
print(X_train.shape)
print(Y_train.shape)
print(X_validate.shape)
print(Y_validate.shape)

(50000, 501, 4)
(50000, 3)
(4999, 501, 4)
(4999, 3)


In [0]:
'''
X_train=array([n[0] for n in sequential_data[0:50000]])
X_validate=array([n[0] for n in sequential_data[50000:55000]])
'''


'\nX_train=array([n[0] for n in sequential_data[0:50000]])\nX_validate=array([n[0] for n in sequential_data[50000:55000]])\n'

In [0]:
'''
Y_train=array([n[1] for n in sequential_data[0:50000]])
Y_validate=array([n[1] for n in sequential_data[50000:55000]])  #Y_test will be used for validation
'''

'\nY_train=array([n[1] for n in sequential_data[0:50000]])\nY_validate=array([n[1] for n in sequential_data[50000:55000]])  #Y_test will be used for validation\n'

In [0]:
from sklearn.preprocessing import StandardScaler
sc_X=StandardScaler()
sc_Y=StandardScaler()

In [0]:
X_train[:,:,0]=sc_X.fit_transform(X_train[:,:,0])
X_train[:,:,1]=sc_X.fit_transform(X_train[:,:,1])
X_train[:,:,2]=sc_X.fit_transform(X_train[:,:,2])
X_train[:,:,3]=sc_X.fit_transform(X_train[:,:,3])

In [0]:
X_validate[:,:,0]=sc_X.fit_transform(X_validate[:,:,0])
X_validate[:,:,1]=sc_X.fit_transform(X_validate[:,:,1])
X_validate[:,:,2]=sc_X.fit_transform(X_validate[:,:,2])
X_validate[:,:,3]=sc_X.fit_transform(X_validate[:,:,3])

In [0]:
Y_train=sc_Y.fit_transform(Y_train)
Y_validate=sc_Y.fit_transform(Y_validate)

In [0]:
                                    #RECURSIVE DEEP NEURAL NETWORK

import torch
import torch.nn as nn
import torchvision.datasets as datasets  #import the dataset
import torchvision.transforms as transforms  #
from torch.autograd import Variable  #autograd is a part of pytorch 0.3 but can also be used with pytorch 0.4
from torch.utils.data import Dataset
import sys

In [0]:
                                    #CONVERT NUMPY TO PYTORCH 
X_train=torch.tensor(X_train)
Y_train=torch.tensor(Y_train)
X_validate=torch.tensor(X_validate)
Y_validate=torch.tensor(Y_validate)

#CONVERT PYTORCH TENSOR TO VARIABLE FORMAT
X_train=Variable(X_train)
Y_train=Variable(Y_train)
X_validate=Variable(X_validate)
Y_validate=Variable(Y_validate)


In [0]:
print(X_train.shape)
print(X_validate.shape)
print(Y_train.shape)
print(Y_validate.shape)

torch.Size([50000, 501, 4])
torch.Size([4999, 501, 4])
torch.Size([50000, 3])
torch.Size([4999, 3])


In [0]:
#WHEN WE ARE USING OUR OWN DATASET THEN WE NEED TO CREATE A DATASET CLASS WHICH INHERITS FROM PREDEFINED 
#DATASET CLASS OF torch.utils.data library
class Dataset(Dataset):
  def __init__(self,x,y):
    self.x=x
    self.y=y
  
  def __getitem__(self,index):
    return self.x[index], self.y[index]
  
  def __len__(self):
    return len(self.x)

In [0]:
train_dataset=Dataset(X_train,Y_train)
validate_dataset=Dataset(X_validate,Y_validate)

In [0]:
print(train_dataset.x.shape)
print(train_dataset.y.shape)
print(validate_dataset.x.shape)
print(validate_dataset.y.shape)

torch.Size([50000, 501, 4])
torch.Size([50000, 3])
torch.Size([4999, 501, 4])
torch.Size([4999, 3])


In [0]:
#MAKE DATA ITERABLE BY LOADING IT TO A LOADER. Shuffe the data to make it independent of order
#Batch size is the number of elements in a batch
train_loader=torch.utils.data.DataLoader(dataset=train_dataset,
                                        batch_size=200,
                                        shuffle=True)

In [0]:
#MAKE DATA ITERABLE BY LOADING IT TO A LOADER. Shuffe the data to make it independent of order
#Batch size is the number of elements in a batch
validation_loader=torch.utils.data.DataLoader(dataset=validate_dataset,
                                        batch_size=200,
                                        shuffle=False)

In [0]:
len(validation_loader)

25

In [0]:
print(train_dataset.x.shape)
print(train_dataset.y.shape)
print(validate_dataset.x.shape)
print(validate_dataset.y.shape)

torch.Size([50000, 501, 4])
torch.Size([50000, 3])
torch.Size([4999, 501, 4])
torch.Size([4999, 3])


In [0]:
#THIS IS ONLY FOR DISPLAY PURPOSE TO UNDERSTAND THE DATA
print("There is {} batches in the dataset".format(len(train_loader)))
shown=0
for (x,y) in train_loader:
  if shown==1:
    break
  print("For one batch there is:")
  print("Data: {}, Type: {} ".format(x.shape,x.dtype))
  print("Data: {}, Type: {} ".format(y.shape,y.dtype))
  shown+=1
  

There is 250 batches in the dataset
For one batch there is:
Data: torch.Size([200, 501, 4]), Type: torch.float64 
Data: torch.Size([200, 3]), Type: torch.float64 


In [0]:
#Input of neural net 2 nodes and output of the neural net is 3 nodes
# Neural Net using 3 hidden layers each containing 3 nodes 

input_size=4
hidden_size=3

class Model(nn.Module):
  def __init__(self):
    super(Model,self).__init__()
    self.bi_rcnn1=torch.nn.LSTM(input_size=4, hidden_size=3, num_layers=1, batch_first=False, bidirectional=True)   
    self.fc1=torch.nn.Linear(6,5)
    self.fc2=torch.nn.Linear(5,5)
    self.fc3=torch.nn.Linear(5,3)
    self.hyperbolic_tan=nn.Tanh()          
    self.relu_nonlinearity=nn.ReLU()
    
    #Input hidden hidden output  =>layer format
  def forward(self,x):
      out,hidden=self.bi_rcnn1(x)  
      out=out[:,250]  #Middle value-1 6 input values to fully connected layer  50 changed to 500
      out=self.hyperbolic_tan(out)
      out=self.fc1(out)
      out=self.relu_nonlinearity(out)
      out=self.fc2(out)
      out=self.relu_nonlinearity(out)
      out=self.fc3(out)
      return out  
  
  def init_hidden(self):
      # Initialize hidden and cell states
      # (num_layers * num_directions, batch, hidden_size)
      return Variable(torch.zeros(1, 200, 3))

    

In [0]:
# Instantiate LSTM model
model = Model()

In [0]:
print(model.parameters)

<bound method Module.parameters of Model(
  (bi_rcnn1): LSTM(4, 3, bidirectional=True)
  (fc1): Linear(in_features=6, out_features=5, bias=True)
  (fc2): Linear(in_features=5, out_features=5, bias=True)
  (fc3): Linear(in_features=5, out_features=3, bias=True)
  (hyperbolic_tan): Tanh()
  (relu_nonlinearity): ReLU()
)>


In [0]:
criterion=torch.nn.MSELoss(size_average=True)   #Loss Criterion

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [51]:
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)
start_time=calendar.timegm(time.gmtime())
epochs=150   #We have done it for 150 epochs 
for epoch in range(epochs):
  for x, y in train_loader:
    x=Variable(x.float())
    y=Variable(y.float())
    outputs=model(x)
    optimizer.zero_grad()
    
    loss=criterion(outputs,y)
    loss.backward()
    optimizer.step()
  
  print("EPOCH {} COMPLETED".format(epoch))
  print("Training loss: {}".format(loss.item()))
end_time=calendar.timegm(time.gmtime())

EPOCH 0 COMPLETED
Training loss: 0.5834723711013794
EPOCH 1 COMPLETED
Training loss: 0.4607582688331604
EPOCH 2 COMPLETED
Training loss: 0.4937290549278259
EPOCH 3 COMPLETED
Training loss: 0.4649192988872528
EPOCH 4 COMPLETED
Training loss: 0.47186997532844543
EPOCH 5 COMPLETED
Training loss: 0.43977832794189453
EPOCH 6 COMPLETED
Training loss: 0.466460645198822
EPOCH 7 COMPLETED
Training loss: 0.45966100692749023
EPOCH 8 COMPLETED
Training loss: 0.3719169795513153
EPOCH 9 COMPLETED
Training loss: 0.43278220295906067
EPOCH 10 COMPLETED
Training loss: 0.3855457603931427
EPOCH 11 COMPLETED
Training loss: 0.4373941421508789
EPOCH 12 COMPLETED
Training loss: 0.3734697103500366
EPOCH 13 COMPLETED
Training loss: 0.3834769129753113
EPOCH 14 COMPLETED
Training loss: 0.4865948557853699
EPOCH 15 COMPLETED
Training loss: 0.46472829580307007
EPOCH 16 COMPLETED
Training loss: 0.3548261523246765
EPOCH 17 COMPLETED
Training loss: 0.36158427596092224
EPOCH 18 COMPLETED
Training loss: 0.398939609527587

In [52]:
end_time=calendar.timegm(time.gmtime())
total_time=(end_time-start_time)/3600
print("FIRST 150 EPOCHS COMPLETED!!!")
print("Time taken to run in hrs is: "+str(total_time))

FIRST 150 EPOCHS COMPLETED!!!
Time taken to run in hrs is: 2.620833333333333


In [53]:
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)
start_time=calendar.timegm(time.gmtime())
epochs=50   #We have done it for 100 epochs 
for epoch in range(epochs):
  for x, y in train_loader:
    x=Variable(x.float())
    y=Variable(y.float())
    outputs=model(x)
    optimizer.zero_grad()
    
    loss=criterion(outputs,y)
    loss.backward()
    optimizer.step()
  
  print("EPOCH {} COMPLETED".format(epoch))
  print("Training loss: {}".format(loss.item()))
end_time=calendar.timegm(time.gmtime())

EPOCH 0 COMPLETED
Training loss: 0.3031535744667053
EPOCH 1 COMPLETED
Training loss: 0.35365116596221924
EPOCH 2 COMPLETED
Training loss: 0.3556913435459137
EPOCH 3 COMPLETED
Training loss: 0.31394773721694946
EPOCH 4 COMPLETED
Training loss: 0.3406875431537628
EPOCH 5 COMPLETED
Training loss: 0.3676520884037018
EPOCH 6 COMPLETED
Training loss: 0.31200334429740906
EPOCH 7 COMPLETED
Training loss: 0.35614219307899475
EPOCH 8 COMPLETED
Training loss: 0.35290008783340454
EPOCH 9 COMPLETED
Training loss: 0.3746221661567688
EPOCH 10 COMPLETED
Training loss: 0.36189666390419006
EPOCH 11 COMPLETED
Training loss: 0.32435905933380127
EPOCH 12 COMPLETED
Training loss: 0.3180472254753113
EPOCH 13 COMPLETED
Training loss: 0.34856128692626953
EPOCH 14 COMPLETED
Training loss: 0.3413362503051758
EPOCH 15 COMPLETED
Training loss: 0.3062087595462799
EPOCH 16 COMPLETED
Training loss: 0.3638611137866974
EPOCH 17 COMPLETED
Training loss: 0.3476991653442383
EPOCH 18 COMPLETED
Training loss: 0.356835335493

In [54]:
total_time=(end_time-start_time)/3600
print("SECOND 100 EPOCHS COMPLETED!!!")
print("Time taken to run in hrs is: "+str(total_time))

SECOND 100 EPOCHS COMPLETED!!!
Time taken to run in hrs is: 0.8672222222222222


In [55]:
count=1
for x, y in validation_loader:
    x=Variable(x.float())
    y=Variable(y.float())
    outputs=model(x)
    
    for i,j in zip(outputs,y):
      print("Predicted "+str(i)+" Actual "+str(j))
    print("Batch: {} completed".format(count))
    count=count+1

Predicted tensor([-0.6144,  0.9027, -0.2240], grad_fn=<SelectBackward>) Actual tensor([-0.4400,  0.3522, -0.4551])
Predicted tensor([-0.3880,  0.4096, -0.0517], grad_fn=<SelectBackward>) Actual tensor([-0.3864,  1.4544, -0.2976])
Predicted tensor([ 0.8937,  0.1553, -0.6090], grad_fn=<SelectBackward>) Actual tensor([ 0.4617, -0.1016, -0.5290])
Predicted tensor([ 0.2295, -0.2942,  0.0307], grad_fn=<SelectBackward>) Actual tensor([ 0.1263,  0.1648, -0.4971])
Predicted tensor([-1.2344, -1.7229,  1.7051], grad_fn=<SelectBackward>) Actual tensor([-0.7051, -1.0531,  2.2837])
Predicted tensor([ 1.0898, -0.2927, -0.4473], grad_fn=<SelectBackward>) Actual tensor([ 0.5092, -0.2104, -0.4995])
Predicted tensor([-0.8916, -0.2185,  0.6067], grad_fn=<SelectBackward>) Actual tensor([-1.7173, -0.7168, -0.6679])
Predicted tensor([-0.0351,  1.5089, -0.9112], grad_fn=<SelectBackward>) Actual tensor([-0.0281,  1.7736, -0.5166])
Predicted tensor([-0.4978,  1.1881, -0.4609], grad_fn=<SelectBackward>) Actual t

In [0]:
outputs=sc_Y.inverse_transform(outputs.detach().numpy())
y=sc_Y.inverse_transform(y.detach().numpy())

In [57]:
#VELOCITY IN ACTUAL FORMAT AFTER INVERSE TRANSFORMATION FOR THE LAST BATCH ie for 200 observations
for i,j in zip(outputs,y):
  print("Predicted "+str(i)+" Actual "+str(j))

Predicted [-0.2226804   0.02943478  6.8944902 ] Actual [ 0.0612526   0.05905173 11.5241785 ]
Predicted [-0.11662736  0.18291615  4.651298  ] Actual [0.17765467 0.10704789 6.0852656 ]
Predicted [0.04239709 0.33124435 2.276492  ] Actual [-0.2685363   0.41925806  0.0579148 ]
Predicted [ 0.4253666  0.6842982 -3.3923814] Actual [1.1916234  0.6599811  0.46826202]
Predicted [0.19502716 0.3819448  1.1050359 ] Actual [0.13236131 0.43548197 0.23221584]
Predicted [1.0508614  0.17954412 0.4186074 ] Actual [0.7344051  0.26783514 0.13266356]
Predicted [-0.30057362 -0.08329322  8.542055  ] Actual [ 0.03029192  0.03001695 11.66539   ]
Predicted [-0.00324359  0.3470067   2.2530482 ] Actual [-0.52329296  0.25980258 -0.25937527]
Predicted [0.03990678 0.22832961 3.5294504 ] Actual [-0.20300956  0.17668952  0.01183874]
Predicted [ 0.6945461  0.481585  -1.9276563] Actual [1.2720736  0.63289905 1.8673404 ]
Predicted [-0.10011222  0.2068171   4.3019757 ] Actual [-0.4333091   0.06441418  0.02801044]
Predicted 

In [58]:
#R-SQUARED METRIC X AXIS
R_residual=((outputs[:,0]-y[:,0])**2).sum()
R_total=((outputs[:,0]-np.average(y[:,0]))**2).sum()
R_squared=1-(R_residual/R_total)
print(R_squared)


0.46744948625564575


In [59]:
#R-SQUARED METRIC Y AXIS
R_residual=((outputs[:,1]-y[:,1])**2).sum()
R_total=((outputs[:,1]-np.average(y[:,1]))**2).sum()
R_squared=1-(R_residual/R_total)
print(R_squared)


0.42005670070648193


In [60]:
#R-SQUARED METRIC Z AXIS
R_residual=((outputs[:,2]-y[:,2])**2).sum()
R_total=((outputs[:,2]-np.average(y[:,2]))**2).sum()
R_squared=1-(R_residual/R_total)
print(R_squared)

0.15023523569107056


In [61]:
#Error difference for 200 values
for i,j in zip(outputs,y):
  print("Error x "+str(i[0]-j[0])+" Error y "+str(i[1]-j[1])+" Error z "+str(i[2]-j[2]))

Error x -0.283933 Error y -0.02961695 Error z -4.6296883
Error x -0.29428202 Error y 0.075868264 Error z -1.4339676
Error x 0.31093338 Error y -0.08801371 Error z 2.2185774
Error x -0.7662568 Error y 0.024317145 Error z -3.8606434
Error x 0.06266585 Error y -0.05353716 Error z 0.8728201
Error x 0.31645626 Error y -0.08829102 Error z 0.28594387
Error x -0.33086553 Error y -0.11331017 Error z -3.123335
Error x 0.5200494 Error y 0.08720413 Error z 2.5124235
Error x 0.24291635 Error y 0.051640093 Error z 3.5176117
Error x -0.5775275 Error y -0.15131405 Error z -3.7949967
Error x 0.33319688 Error y 0.14240292 Error z 4.2739654
Error x 0.19319814 Error y 0.10132326 Error z 0.5174886
Error x 0.22830981 Error y 0.09705929 Error z 0.8361075
Error x -0.38847315 Error y 0.052729867 Error z 0.45161295
Error x 0.14906836 Error y -0.08480855 Error z 0.8266729
Error x 0.0066707134 Error y -0.08847742 Error z 0.399184
Error x 0.33722672 Error y -0.029492855 Error z 0.45448267
Error x -0.3924448 Error 

In [62]:
#Error difference for 200 values
for i,j in zip(outputs,y):
  print("Error x "+str((i[0]-j[0])*100/j[0])
        +" Error y "+str((i[1]-j[1])*100/j[1])+" Error z "+str((i[2]-j[2])*100/j[2]))

Error x -463.5444160422129 Error y -50.15424963711813 Error z -40.17369447161311
Error x -165.64834536033317 Error y 70.87320162887316 Error z -23.56458494998365
Error x -115.78821259401306 Error y -20.992729155017862 Error z 3830.7606153610236
Error x -64.3036027587664 Error y 3.684521629655787 Error z -824.4622131524504
Error x 47.34453821435732 Error y -12.29377204898596 Error z 375.8658720008244
Error x 43.09014985708051 Error y -32.96468832507553 Error z 215.54061984964227
Error x -1092.2565677744244 Error y -377.4873043985046 Error z -26.77437171600373
Error x -99.38016258288295 Error y 33.56553600974955 Error z -968.6441867866839
Error x -119.65758891041749 Error y 29.226460735641517 Error z 29712.714273161222
Error x -45.40047926482863 Error y -23.908086278347803 Error z -203.2300402179798
Error x -76.89588616233596 Error y 221.07386191121998 Error z 15258.47526943099
Error x 20.033008793424397 Error y 321.1720766365794 Error z 694.416376418966
Error x 23.397165038393275 Error 

In [63]:
o=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
print(o[2:12:3])

[3, 6, 9, 12]


In [64]:
11%11

0